## 음악차트 데이터 크롤링

안녕하세요? 강사 이천입니다.

2강 음악차트 데이터 크롤링에서는, 원하는 음악 사이트에 웹브라우저를 통해 접속하여,

원하는 데이터를 다운받아서 텍스트 형태로 변환하는 작업을 해 보도록 할게요!!

(코딩을 하실 때는 되도록 ctrl+c,v를 쓰지 마세요!! 실력이 늘지 않습니다!!)

그럼 시작해 볼까요? :)


필요한 모듈을 먼저 불러옵니다!!

(만약 에러가 난다면, !pip install selenium)과 같이 아래에 입력하여 필요한 모듈을 먼저 설치합니다!!

In [5]:
!pip install selenium
!pip install time

You should consider upgrading via the 'c:\anaconda\python.exe -m pip install --upgrade pip' command.


ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
You should consider upgrading via the 'c:\anaconda\python.exe -m pip install --upgrade pip' command.


In [6]:
import selenium
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import repeat
from selenium import webdriver

이제 크롤링에 사용할 브라우저를 셋팅합니다.

주로 사용하는 브라우저 외에, 추가로 브라우저를 설치해 주세요!!

예를 들어, 저는 Firefox를 잘 사용하지 않기 때문에 Firefox를 새로 설치하고,

Firefox를 통해 크롤링할 수 있는 'geckodriver.exe'를 다운로드하였습니다.

(geckodriver 다운로드: https://github.com/mozilla/geckodriver/releases)

주로 사용하는 브라우저를 사용하실 경우, 크롤링 및 자동화 작업 중 

원래 본인이 사용하던 자료를 건드릴 우려가 있으니, 가급적 피해주세요!! ^^

* Firefox의 브라우저 정보 - 정보 에 들어가서 Firefox 버전을 찾아서 적어 둡니다.

예) 94.0.1 (64-비트)

이 브라우저의 버전에 맞는 geckodriver를 다운받습니다.

(geckodriver 다운로드: https://github.com/mozilla/geckodriver/releases)


그리고, music_data_02가 들어 있는 폴더에 같이 넣어주세요!!

저의 경우, c:\users\student\desktop\music-data\에 geckodriver.exe를 
    
복사해 주었습니다!!

이제 geckodriver를 실행해 줍니다.

아래 코드를 실행하면, 브라우저가 

다른 창에서 실행되고, 창 크기가 최대로 확대됩니다. ^^

In [7]:
# gecko드라이버 열기
driver = webdriver.Firefox() 
# geckodriver 경로 추가
driver.maximize_window() 
# 창 크기 최대


이제, 브라우저로 하여금 음악차트 페이지에 접속하도록 url을 넣어 줍니다.

드라이버가 사용자의 명령에 따라, url으로 접속합니다.

In [8]:
# 드라이버가 해당 url 접속
url = 'https://www.melon.com/chart/index.htm' # 멜론차트 페이지
driver.get(url)

차례대로 웹 화면을 브라우저가 클릭하여, 원하는 메뉴로 접근합니다.

브라우저에서 CTRL+SHIFT+I 를 누르면,

웹 개발자 화면이 열립니다.




개발자 화면에서, 브라우저의 상대 요소(xPath)를 가져오는 방법을 알아봅시다!!

(월간차트 xpath:)
/html/body/div/div[3]/div/div/form/div[1]/div/h4[2]/a

In [9]:
# 차트파인더 클릭
driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span').click()

# 연대선택, 연도선택, 월선택, 장르선택

# 월간차트 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[2]/a').click()
time.sleep(2)

# 연대선택 2000년 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[3]/span/label').click()
time.sleep(2)

# 연도선택 2008년 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[2]/span/label').click()
time.sleep(2)

# 월선택 8월 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[8]/span/label').click()
time.sleep(2)

# 장르선택 종합 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]/span/label').click()
time.sleep(2)

# 검색버튼 클릭
driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span').click()

열려 있는 페이지에서, 전체 HTML을 다운받아 옵니다.

page_source = 전체 HTML 

--> html 변수에 저장

--> soup : HTML을 자동으로 분석하기 위해, BeautifulSoup이라는 모듈을 사용합니다.
(이후 설명)

In [10]:
html = driver.page_source # 드라이버 현재 페이지의 html 정보 가져오기 
                            # cf) requests.get(url)
soup = BeautifulSoup(html, 'lxml')

이제, html이 soup이라는 변수에 저장되어 있습니다.

[수프 그릇에 html이 담겨 있는 것을 상상하시면 쉬워요!! ^^]

이 수프(soup)에서, 자신이 원하는 것을 건져 내는 방법을 알아보도록 할게요!!

만일 아래 부분을 가져오고자 한다면


<div class="ellipsis rank01"> (# --> 이 부분을 주목해 주세요!!)
    <span><strong>
        <a href="javascript:melon.play.playSong('19080101','1903050');" title="U-Go-Girl (With 낯선) 재생 - 새창">U-Go-Girl (With 낯선)</a>
        </strong>
    </span>
</div>

soup에서 해당 부분을 조건문을 걸어 찾아 줍니다.

HTML파일에 있는 <div> 요소 중, 'class = ellipsis rank01'이라고 쓰여진 것만을 
    
찾고자 한다면,

In [11]:
soup.find_all('div', attrs={'class': 'ellipsis rank01'})


[<div class="ellipsis rank01"><span>
 <strong><a href="javascript:melon.play.playSong('19080101','1903050');" title="U-Go-Girl (With 낯선) 재생 - 새창">U-Go-Girl (With 낯선)</a></strong>
 </span></div>,
 <div class="ellipsis rank01"><span>
 <strong><a href="javascript:melon.play.playSong('19080101','1897011');" title="사랑과 전쟁 (Narr. 하하) 재생 - 새창">사랑과 전쟁 (Narr. 하하)</a></strong>
 </span></div>,
 <div class="ellipsis rank01"><span>
 <strong><a href="javascript:melon.play.playSong('19080101','1906673');" title="신데렐라 재생 - 새창">신데렐라</a></strong>
 </span></div>,
 <div class="ellipsis rank01"><span>
 <strong><a href="javascript:melon.play.playSong('19080101','1913501');" title="하루 하루 재생 - 새창">하루 하루</a></strong>
 </span></div>,
 <div class="ellipsis rank01"><span>
 <strong><a href="javascript:melon.play.playSong('19080101','1874397');" title="So Hot 재생 - 새창">So Hot</a></strong>
 </span></div>,
 <div class="ellipsis rank01"><span>
 <strong><a href="javascript:melon.play.playSong('19080101','1890405');" tit

위에서 조건문을 걸어 낸 'div', attrs={'class': 'ellipsis rank01'}

중에서 'title'부분만을 추출하고자 한다면,


In [12]:
[title.find('a').get_text() for title in soup.find_all('div', attrs={'class': 'ellipsis rank01'})]

['U-Go-Girl (With 낯선)',
 '사랑과 전쟁 (Narr. 하하)',
 '신데렐라',
 '하루 하루',
 'So Hot',
 '가지마 가지마',
 '사랑해 미안해',
 'D.I.S.C.O  (Feat. TOP)',
 '이노래',
 '사랑을 원해',
 '에너지 (Feat. 선예)',
 '너 때문에',
 '누난 너무 예뻐 (Replay)',
 '나만 바라봐',
 'Hey Mr. Big',
 'Blue Moon',
 'Moai',
 '천국',
 '안녕 나의 사랑',
 'Beautiful Life',
 '못된 여자 (Feat. 서인영)',
 'No.3 (Feat. Sol Flower)',
 '발자국',
 'Too Much (With 서인영) (우리결혼했어요 화제곡)',
 '나만 바라봐 (Part 2)',
 '그대에게 바래요',
 '가시 물고기',
 'Oh My Friend (Feat. 노브레인)',
 '착한 사람',
 '라라라',
 '그대라면',
 '사랑같은건 없나봐 (우리 결혼했어요 삽입곡)',
 '서커스 (Feat. 임유경 - 달래 음악단 , ＄howgun)',
 'Honey',
 'Rock U',
 'Lie',
 '죽도록 사랑해 (Feat. 박정현)',
 'Lady',
 '그럴꺼면 왜 (Feat. 베이지)',
 '가슴으로 외쳐 (Rap Feat. Rhymer)',
 '날 봐, 귀순',
 '우산 (Feat. 윤하)',
 '미쳐미쳐미쳐',
 'This Time',
 '이순간 이대로',
 '미치겠어 (Feat. M.A.C)',
 'One (Feat. 지선)',
 '사랑해요',
 '헤픈여자',
 'Kiss',
 '화신 (花信)',
 '화분 (알렉스 Ver.) (우리결혼했어요 삽입곡)',
 '그대만 보며',
 'Intro (Stand Up)',
 '그대만 보여요 (Feat. BigBang T.O.P)',
 "그대만이 (MBC '우리 결혼했어요' 알렉스&신애 테마곡)",
 'Bad Boy',
 '6년동안',
 '추억속의 그대',
 '행복해야 해',
 '천하무적

In [13]:

soup.find_all('span', attrs={'class':'checkEllipsis'})


[<span class="checkEllipsis" style="display:none"><a class="fc_mgray" href="javascript:melon.link.goArtistDetail('1713');" title="이효리 - 페이지 이동">이효리</a></span>,
 <span class="checkEllipsis" style="display:none"><a class="fc_mgray" href="javascript:melon.link.goArtistDetail('236815');" title="다비치 - 페이지 이동">다비치</a></span>,
 <span class="checkEllipsis" style="display:none"><a class="fc_mgray" href="javascript:melon.link.goArtistDetail('26265');" title="서인영 - 페이지 이동">서인영</a></span>,
 <span class="checkEllipsis" style="display:none"><a class="fc_mgray" href="javascript:melon.link.goArtistDetail('198094');" title="BIGBANG - 페이지 이동">BIGBANG</a></span>,
 <span class="checkEllipsis" style="display:none"><a class="fc_mgray" href="javascript:melon.link.goArtistDetail('217764');" title="원더걸스 - 페이지 이동">원더걸스</a></span>,
 <span class="checkEllipsis" style="display:none"><a class="fc_mgray" href="javascript:melon.link.goArtistDetail('100742');" title="브라운 아이즈 - 페이지 이동">브라운 아이즈</a></span>,
 <span class=

창작자명만을 조건문을 걸어 찾기 위해, 아래와 같이 만들어 줍니다.


In [14]:
[ singer.get_text() for singer in soup.find_all('span', attrs={'class':'checkEllipsis'}) ]

['이효리',
 '다비치',
 '서인영',
 'BIGBANG',
 '원더걸스',
 '브라운 아이즈',
 '지아',
 '엄정화',
 '2am',
 '쿨 (COOL)',
 '마이티 마우스',
 '브라운 아이즈',
 'SHINee (샤이니)',
 '태양',
 '이효리',
 '씨야, 다비치, 블랙펄',
 '서태지',
 'BIGBANG',
 '성시경',
 'V.O.S',
 '원투',
 '배치기',
 '먼데이 키즈 (Monday Kiz)',
 '크라운제이 (CROWN J)',
 'G-DRAGON',
 '솔비, 김종욱',
 '엠씨더맥스 (M.C the MAX)',
 'BIGBANG',
 'BIGBANG',
 'SG 워너비',
 '알렉스',
 '가비엔제이',
 'MC몽',
 '옥주현',
 '카라',
 '가비엔제이',
 'MC몽',
 'BIGBANG',
 'PK 헤만',
 '먼데이 키즈 (Monday Kiz)',
 '대성',
 '에픽하이 (EPIK HIGH)',
 '바나나 걸',
 '원더걸스',
 '브라운 아이즈',
 'MC몽',
 '에픽하이 (EPIK HIGH)',
 '이수 (엠씨더맥스)',
 '일 락',
 '김건모',
 '박효신',
 '알렉스',
 '이정',
 'BIGBANG',
 '지아',
 '김종욱',
 '손담비',
 '별',
 '이승기',
 '이기찬',
 '이효리',
 '남규리',
 '박명수',
 '나몰라패밀리 (Namolla Family)',
 '김동률',
 '서태지',
 '신지',
 '브라운 아이즈',
 '란 (RAN)',
 'SG 워너비',
 '정철 (최정철)',
 '미노 (프리스타일), 타우',
 '노을',
 '넬 (NELL)',
 '서태지',
 '이효리',
 '이효리',
 '이효리',
 '김범수',
 '쥬얼리',
 '제드',
 'Jennifer Lopez',
 'BMK',
 '란 (RAN)',
 '플라이 투 더 스카이',
 '쥬얼리',
 'SS501',
 '다비치',
 '다이나믹 듀오',
 '손담비',
 'BIGBANG',
 '제이워크',
 '이효리',
 '

순위 정보를 조건문을 걸어 가져와 봅니다!!

<span class="rank top">1</span>

In [15]:
soup.find_all('span', attrs={'class':'rank top'})


[<span class="rank top">1</span>,
 <span class="rank top">2</span>,
 <span class="rank top">3</span>]

순위 정보의 경우, 1,2,3위와 4-100위의 순위가

정확하게 표시되지 않습니다.

따라서, 긁어온 순서에 따라 '새로' 순위 정보를 만들어 줍니다. ^^

크롤링한 순서에 따라 'rank'에 새로 1~끝까지

순위 점수를 매겨 주었습니다.

In [16]:
song = [title.find('a').get_text() for title in soup.find_all('div', attrs={'class': 'ellipsis rank01'})]
rank = []
for i in range(len(song)):
    rank.append(i+1)

<span class="datelk"> 08</span>

In [17]:
soup.find_all('span', attrs={'class':'datelk'})


[<span class="datelk">2008</span>, <span class="datelk"> 08</span>]

제목의 '2008년 08월종합월간차트' 부분을 가져옵니다.

파싱하여 분석!!

In [18]:
soup.find_all('span', attrs={'class':'datelk'})[0].get_text() # 년
soup.find_all('span', attrs={'class':'datelk'})[1].get_text() # 월

' 08'

selenium을 통해 월간차트 클릭 -> 연대 선택 -> 연도 선택 -> 월 선택 -> 장르 선택을 하였습니다.

2020년 8월과 2008년 8월의 차이는 연대와 연도이므로, 이 두 부분의 규칙성을 파악해 봅시다. 



연대 xpath 규칙성
2000년 //*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[3]/span/label
2010년 //*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[2]/span/label

2020년 //*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[1]/span/label

연도 xpath 규칙성
2009년 //*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[1]/span/label
2008년 //*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[2]/span/label
2020년 //*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[2]/span/label

2008년과 2020년 모두 위에서 2번째라 xpath 같음!

추가) 월 xpath 규칙성
1월 //*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[1]/span/label
12월 //*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[12]/span/label


In [19]:
period = 1
month = 8
result_df = pd.DataFrame() #크롤링한 결과를 result_df에 저장합니다!!

while period < 4:
    try:
        # 크롬드라이버 열기
        driver = webdriver.Firefox() # gecko드라이버 경로
        driver.maximize_window() # 브라우저창 크기 최대

        # 드라이버가 해당 url 접속
        url = 'https://www.melon.com/chart/index.htm' # 멜론차트 페이지
        driver.get(url)
        time.sleep(2)

        # 차트파인더 클릭
        driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span').click()
        time.sleep(2)

        # 월간차트 클릭
        driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[2]/a').click()
        time.sleep(2)

        ## 여기에서 period 사용
        driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[{}]/span/label'.format(period)).click()
        time.sleep(2)

        # 연도선택(규칙 찾기!)
        driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[2]/span/label').click()
        time.sleep(2)

        # 월선택 8월 클릭
        driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[{}]/span/label'.format(month)).click()
        time.sleep(2)

        # 장르선택 종합 클릭
        driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]/span/label').click()
        time.sleep(2)
        
        # 검색버튼 클릭
        driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span').click()
        time.sleep(2)
        
        # html 정보 가져오기
        html = driver.page_source 
        soup = BeautifulSoup(html, 'lxml')
        
        # 노래 제목 가져오기
        song_list = [title.find('a').get_text() for title in soup.find_all('div', attrs={'class': 'ellipsis rank01'})]
        
        # 가수명 가져오기
        singer_list = [ singer.get_text() for singer in soup.find_all('span', attrs={'class':'checkEllipsis'}) ]
        
        # 순위 만들기
        rank_list = []
        for i in range(len(song)):
            rank_list.append(i+1)
        # 년
        year_list = list(repeat(soup.find_all('span', attrs={'class':'datelk'})[0].get_text(), len(song_list)))
        
        # 월 
        # month = list(repeat(soup.find_all('span', attrs={'class':'datelk'})[1].get_text(), len(song))) # 08로 표기되어 안깔끔
        month_list = list(repeat(month, len(song_list)))
        
        # 데이터프레임 생성
        df = pd.DataFrame({'연도':year_list,'월':month_list,'순위':rank_list,'곡명':song_list,'가수명':singer_list})
        result_df = pd.concat([result_df, df], ignore_index=True)
        period += 2
    
    except:
        print(period)
        break

드라이버 창을 닫아 줍니다!!

In [21]:
# driver.close()

분석결과를 .csv파일로 내보내기합니다.

(파일이 깨질 경우, encoding = 부분을 'ANSI', 'EUC-KR", 'KO-KR', 'UTF-8'등으로
변환해주세요!!

In [23]:
filename = 'music_data_08_08.csv'
result_df.to_csv(filename, encoding='ANSI')
print('file exported to : ',filename)

file exported to :  music_data_08_08.csv


수고하셨습니다!! :D

------------------------------------------------------------------------------

Reference: https://arehoow.tistory.com/10

본 수업 및 수업자료는 학생들의 데이터 수집을 돕기 위한 자료이며, 

크롤링 대상의 지적재산권 및 이 소프트웨어의 실제 사용에 의해 발생하는 제반 문제에 대하여 강사는 일절 책임지지 않습니다.